# COCO Stop Sign Dataset
- Download COCO train, test, val dataset using fifty one library
https://cocodataset.org/#download
- Generate binary segmentation mask based on "stop sign" and "background"

In [1]:
import os
if not os.path.exists("dataset"): 
    os.mkdir("dataset")

In [2]:
import fiftyone.zoo as foz
import fiftyone as fo

In [11]:
fo.config.dataset_zoo_dir = "./dataset"

In [ ]:
# Downloading coco dataset
# Referenced from https://medium.com/voxel51/the-coco-dataset-best-practices-for-downloading-visualization-and-evaluation-68a3d7e97fb7

train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["segmentations"],
    classes=["stop sign"],
    max_samples=2000,
)


val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=["stop sign"],
    max_samples=100,
)


Found annotations at 'dataset/coco-2017/raw/instances_train2017.json'


Only found 1734 (<2000) samples matching your requirements
 100% |████████████████| 1734/1734 [32.6s elapsed, 0s remaining, 54.5 images/s]      
Writing annotations for 1734 downloaded samples to './dataset/coco-2017/train/labels.json'
Dataset info written to './dataset/coco-2017/info.json'
Loading existing dataset 'coco-2017-train-2000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found annotations at 'dataset/coco-2017/raw/instances_val2017.json'
Only found 69 (<100) samples matching your requirements
 100% |████████████████████| 69/69 [2.0s elapsed, 0s remaining, 39.1 images/s]         
Writing annotations for 69 downloaded samples to './dataset/coco-2017/validation/labels.json'
Dataset info written to './dataset/coco-2017/info.json'
Loading existing dataset 'coco-2017-validation-100'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
from collections import defaultdict
import json
import numpy as np

# Referenced from https://machinelearningspace.com/coco-dataset-a-step-by-step-guide-to-loading-and-visualizing/
class COCOParser:
    def __init__(self, anns_file, imgs_dir):
        with open(anns_file, 'r') as f:
            coco = json.load(f)
            
        self.annIm_dict = defaultdict(list)        
        self.cat_dict = {} 
        self.annId_dict = {}
        self.im_dict = {}
        self.licenses_dict = {}
        for ann in coco['annotations']:           
            self.annIm_dict[ann['image_id']].append(ann) 
            self.annId_dict[ann['id']]=ann
        for img in coco['images']:
            self.im_dict[img['id']] = img
        for cat in coco['categories']:
            self.cat_dict[cat['id']] = cat
        for license in coco['licenses']:
            self.licenses_dict[license['id']] = license
    def get_imgIds(self):
        return list(self.im_dict.keys())
    def get_annIds(self, im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        return [ann['id'] for im_id in im_ids for ann in self.annIm_dict[im_id]]
    def load_anns(self, ann_ids):
        im_ids=ann_ids if isinstance(ann_ids, list) else [ann_ids]
        return [self.annId_dict[ann_id] for ann_id in ann_ids]        
    def load_cats(self, class_ids):
        class_ids=class_ids if isinstance(class_ids, list) else [class_ids]
        return [self.cat_dict[class_id] for class_id in class_ids]
    def get_imgLicenses(self,im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        lic_ids = [self.im_dict[im_id]["license"] for im_id in im_ids]
        return [self.licenses_dict[lic_id] for lic_id in lic_ids]

In [ ]:
anns_file = "/dataset/coco-2017/validation/labels.json"
imgs_dir = "/dataset/coco-2017/validation/data"
masks_dir = "/root/18794/18794_RoadVision/dataset/coco-2017/validation/masks"

In [ ]:
parser = COCOParser(anns_file, imgs_dir)

In [ ]:
len(parser.im_dict)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import os

stop_sign_category = 13

def visualize_with_mask(parser, img_id, imgs_dir, masks_dir, save):
    img_info = parser.im_dict[img_id]
    img_path = os.path.join(imgs_dir, img_info['file_name'])

    with Image.open(img_path) as img:
        width, height = img.size

    # Create an empty binary mask image
    mask = Image.new("1", (width, height), 0) 
    draw = ImageDraw.Draw(mask)
    
    ann_ids = parser.get_annIds([img_id])
    anns = parser.load_anns(ann_ids)
    
    for ann in anns:
        if ann['category_id'] == stop_sign_category and 'segmentation' in ann:
            for seg in ann['segmentation']:
                polygon = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                draw.polygon(polygon, fill=1) 
                
    plt.imshow(np.array(mask), cmap="gray")
    plt.axis("off")
    print(img_info["file_name"])
    # print("IMAGE:", width, height)
    # print("MASK:", np.array(mask).shape)
    if not save:
        plt.show()
    else:
        mask = mask.convert("L")  # Convert to grayscale 
        save_path = os.path.join(masks_dir, f"mask_{img_info['file_name']}")
        mask.save(save_path)
    plt.clf()


In [ ]:
img_ids = parser.get_imgIds()
visualize_with_mask(parser, img_id=img_ids[0], imgs_dir=imgs_dir, masks_dir=masks_dir, save=True)

In [ ]:
from PIL import Image
import numpy as np
import cv2

image = Image.open("/root/18794/18794_RoadVision/dataset/coco-2017/validation/masks/mask_000000297343.jpg")
image_array = np.array(image)
image_array.shape

## Generate binary image segmentation masks

In [13]:
for ds_type in ["train", "validation"]:
    anns_file = f"/dataset/coco-2017/{ds_type}/labels.json"
    imgs_dir = f"/dataset/coco-2017/{ds_type}/data"
    masks_dir = f"/root/18794/18794_RoadVision/dataset/coco-2017/{ds_type}/masks"

    parser = COCOParser(anns_file, imgs_dir)

    for id in parser.im_dict.keys():
        visualize_with_mask(parser, img_id=id, imgs_dir=imgs_dir, masks_dir=masks_dir, save=True)

    print(f"{ds_type} dataset completed")


000000483108.jpg
IMAGE: 428 640
MASK: (640, 428)
000000334405.jpg
IMAGE: 640 494
MASK: (494, 640)
000000158952.jpg
IMAGE: 640 424
MASK: (424, 640)
000000308599.jpg
IMAGE: 375 500
MASK: (500, 375)
000000249720.jpg
IMAGE: 500 333
MASK: (333, 500)
000000463633.jpg
IMAGE: 640 480
MASK: (480, 640)
000000383470.jpg
IMAGE: 640 480
MASK: (480, 640)
000000578292.jpg
IMAGE: 480 640
MASK: (640, 480)
000000171500.jpg
IMAGE: 640 426
MASK: (426, 640)
000000440329.jpg
IMAGE: 640 427
MASK: (427, 640)
000000098590.jpg
IMAGE: 640 478
MASK: (478, 640)
000000488327.jpg
IMAGE: 403 640
MASK: (640, 403)
000000142129.jpg
IMAGE: 640 427
MASK: (427, 640)
000000319345.jpg
IMAGE: 640 480
MASK: (480, 640)
000000024343.jpg
IMAGE: 425 640
MASK: (640, 425)
000000214698.jpg
IMAGE: 640 479
MASK: (479, 640)
000000199339.jpg
IMAGE: 428 640
MASK: (640, 428)
000000271240.jpg
IMAGE: 425 640
MASK: (640, 425)
000000147915.jpg
IMAGE: 427 640
MASK: (640, 427)
000000189427.jpg
IMAGE: 500 333
MASK: (333, 500)
000000493503.jpg
IMA

<Figure size 640x480 with 0 Axes>

In [ ]:
from PIL import Image
import numpy as np
import cv2

# Load the image using Pillow
image = Image.open("/root/18794/18794_project/datasets/data/coco/train/data/000000000064.jpg")
mask = Image.open("/root/18794/18794_project/datasets/data/coco/train/masks/mask_000000000064.jpg")

# Convert the image to a NumPy array
image_array = np.array(image)
mask_array = np.array(mask)
height, width = image_array.shape[:2]
mask_resized = cv2.resize(mask_array, (width, height), interpolation=cv2.INTER_NEAREST)


# Display the size of the image
print("Image size (Height, Width, Channels):", image_array.shape)
print("Mask size (Height, Width, Channels):", mask_resized.shape)

# Overlay the mask on the image
# Normalize mask to range [0, 1] if not already
mask_resized = mask_resized / 255.0 if mask_resized.max() > 1 else mask_resized

# Apply the mask as an overlay with transparency
overlay = image_array.copy()


In [ ]:
overlay[mask_resized > 0] = 0

In [ ]:

# overlay[mask_resized > 0] = [255,0,0]  # Make mask regions red

# Plot the result
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image_array)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Overlay: Mask on Image")
plt.imshow(overlay)
plt.axis("off")

plt.tight_layout()
plt.show()
 